In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
using QuantumOptics

In [3]:
function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [28]:
Nx = 6; Ny = 6; N=Nx*Ny; q = 6; N_cut = 6
PN = 3
U = 2

2

In [29]:
using LinearAlgebra

basis = NLevelBasis(N)

NLevel(N=36)

In [30]:
H1 = Hofstadter_SP_Op(Nx, Ny, 1/q, basis)

Operator(dim=36x36)
  basis: NLevel(N=36)sparse([2, 6, 7, 31, 1, 3, 8, 32, 2, 4  …  33, 35, 5, 29, 34, 36, 6, 30, 31, 35], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  34, 34, 35, 35, 35, 35, 36, 36, 36, 36], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -0.5000000000000001 - 0.8660254037844386im, -0.5000000000000001 + 0.8660254037844386im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 0.5000000000000004 + 0.8660254037844386im, 0.5000000000000004 - 0.8660254037844386im, -1.0 + 0.0im, -1.0 + 0.0im, -0.49999999999999933 + 0.866025403784439im, -0.49999999999999933 - 0.866025403784439im, -1.0 + 0.0im, -1.0 + 0.0im], 36, 36)

In [31]:
eigenenergies(dense(H1))

36-element Vector{Float64}:
 -3.0955735647785607
 -3.09557356477856
 -3.09557356477856
 -3.095573564778559
 -3.0955735647785585
 -3.0955735647785567
 -1.4142135623730958
 -1.4142135623730956
 -1.4142135623730945
 -1.4142135623730943
 -1.414213562373094
 -1.414213562373092
 -0.6460838219953846
  ⋮
  1.4142135623730916
  1.4142135623730934
  1.414213562373094
  1.4142135623730945
  1.4142135623730967
  1.4142135623730983
  3.095573564778559
  3.095573564778559
  3.0955735647785603
  3.0955735647785607
  3.0955735647785607
  3.0955735647785616

In [32]:
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [33]:
function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [34]:
sub_states = get_sub_states(H1, N_cut);

In [35]:
function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [36]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [37]:
function Subspace_Op(SP_Op, P)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [38]:
H1_sub = Subspace_Op(H1, P)

Operator(dim=6x6)
  basis: Subspace(superbasis=NLevel(N=36), states:6)
 -3.09557-1.0e-17im  -7.0e-17+1.6e-16im  …   2.4e-16-1.7e-16im
 -6.0e-17-1.0e-16im  -3.09557-4.0e-17im     -2.1e-16+2.9e-16im
  6.0e-17+4.5e-16im   2.8e-16+2.2e-16im     -2.1e-16-7.0e-17im
 -1.0e-17+1.0e-17im   1.9e-16-1.2e-16im      1.8e-16+1.2e-16im
 -1.1e-16-5.0e-17im   6.0e-17+1.1e-16im     -8.0e-17+4.0e-17im
  1.7e-16+1.0e-16im  -4.7e-16+2.2e-16im  …  -3.09557-3.0e-17im

In [39]:
num_sub_list = []
for m in 1:N
    NM = transition(basis, m, m)
    NMP = Subspace_Op(NM, P)
    push!(num_sub_list, NMP)
end

In [40]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=36), states:6), states:56)

In [41]:
function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub)

Operator(dim=56x56)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=36), states:6), states:56)sparse([1, 2, 3, 4, 5, 6, 1, 2, 3, 4  …  52, 54, 55, 56, 21, 36, 46, 52, 55, 56], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  55, 55, 55, 55, 56, 56, 56, 56, 56, 56], ComplexF64[-9.286720694335678 - 2.0e-17im, -1.0e-16 - 1.7e-16im, 1.0e-16 + 7.8e-16im, -3.0e-17 + 2.0e-17im, -1.9e-16 - 8.0e-17im, 2.9e-16 + 1.8e-16im, -1.2e-16 + 2.8e-16im, -9.286720694335681 - 5.0e-17im, 2.8e-16 + 2.2e-16im, 1.9e-16 - 1.2e-16im  …  -2.2e-16 + 1.3e-16im, -1.7e-16 + 8.0e-17im, -9.286720694335683 - 6.0e-17im, -1.4e-16 - 1.3e-16im, 4.2e-16 - 3.0e-16im, -3.6e-16 + 5.1e-16im, -3.7e-16 - 1.2e-16im, 3.1e-16 + 2.2e-16im, -1.4e-16 + 7.0e-17im, -9.286720694335681 - 8.0e-17im], 56, 56)

In [42]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [43]:
num_mb_list = []
for m in 1:N
    NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
    push!(num_mb_list, NMP)
end

In [44]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U)

Operator(dim=56x56)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=36), states:6), states:56)sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  40, 43, 45, 46, 49, 51, 52, 54, 55, 56], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  56, 56, 56, 56, 56, 56, 56, 56, 56, 56], ComplexF64[-2.1246964363080467 + 0.0im, -0.02713649406611147 + 0.00260134484177779im, 0.03016980722124858 - 0.02644308806051652im, -0.03107780643415557 - 0.02251688722903574im, 0.00647762305856232 + 0.00706275981596395im, 0.11094834864570895 - 0.01818147219863103im, -0.00597637972544214 + 0.0115867177834336im, 0.00115253264394028 + 0.03348915968759518im, -0.0062007387668364 + 0.02282777299233117im, 0.00270337216286825 - 0.03311952549295624im  …  0.01548843769032503 + 0.01654206505977928im, -0.00717359839885605 + 0.0104645315117879im, -0.01532111663763253 + 0.00227859739951872im, -0.06461151884229391 + 0.01411477837222533im, 0.00135068727799435 + 0.00411319026635876im, -0.00200374807226321 + 0.0003326274795068im, 0.009914628810

In [45]:
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)

Operator(dim=56x56)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=36), states:6), states:56)
    0.375304+0.0im         …          0.0+0.0im
  -0.0271365+0.00260134im             0.0+0.0im
   0.0301698-0.0264431im              0.0+0.0im
  -0.0310778-0.0225169im              0.0+0.0im
  0.00647762+0.00706276im             0.0+0.0im
    0.110948-0.0181815im   …    0.0280014+0.0132819im
 -0.00597638+0.0115867im              0.0+0.0im
  0.00115253+0.0334892im              0.0+0.0im
 -0.00620074+0.0228278im              0.0+0.0im
  0.00270337-0.0331195im              0.0+0.0im
  -0.0096087+0.0212551im   …    0.0160035-0.0246312im
  0.00417255+0.00867813im             0.0+0.0im
  -0.0116291+0.0222003im              0.0+0.0im
            ⋮              ⋱             ⋮
         0.0+0.0im             -0.0153211+0.0022786im
         0.0+0.0im         …   -0.0646115+0.0141148im
         0.0+0.0im                    0.0+0.0im
         0.0+0.0im                    0.0+0.0im
         0

In [46]:
H_MB = H1_MB + H_Int_MB
H_mb = H1_MB + Vint_mb

Operator(dim=56x56)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=36), states:6), states:56)
    -8.91142-2.0e-17im     …          0.0+0.0im
  -0.0271365+0.00260134im             0.0+0.0im
   0.0301698-0.0264431im              0.0+0.0im
  -0.0310778-0.0225169im              0.0+0.0im
  0.00647762+0.00706276im             0.0+0.0im
    0.110948-0.0181815im   …    0.0280014+0.0132819im
 -0.00597638+0.0115867im              0.0+0.0im
  0.00115253+0.0334892im              0.0+0.0im
 -0.00620074+0.0228278im              0.0+0.0im
  0.00270337-0.0331195im              0.0+0.0im
  -0.0096087+0.0212551im   …    0.0160035-0.0246312im
  0.00417255+0.00867813im             0.0+0.0im
  -0.0116291+0.0222003im              0.0+0.0im
            ⋮              ⋱             ⋮
         0.0+0.0im             -0.0153211+0.0022786im
         0.0+0.0im         …   -0.0646115+0.0141148im
         0.0+0.0im                    0.0+0.0im
         0.0+0.0im                    0.0+0.0im
         0

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [47]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [48]:
# using Plots

E_MB = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))

E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E_MB,'\n', E_mb,'\n', E2[1:size(H_mb,1)])

[-11.783305722874747, -11.783305722874742, -11.673307821272257, -11.673307821272244, -11.673307821272244, -11.673307821272243, -11.67330782127224, -11.67330782127224, -11.673307821272234, -11.67330782127223, -11.63244877346422, -11.632448773464214, -11.632448773464208, -11.632448773464207, -11.632448773464205, -11.632448773464205, -11.632448773464203, -11.6324487734642, -11.575920984385268, -11.575920984385263, -11.569972566045514, -11.569972566045509, -11.569972566045507, -11.569972566045507, -11.569972566045502, -11.569972566045502, -11.5699725660455, -11.569972566045495, -11.528040246290535, -11.528040246290534, -11.528040246290532, -11.528040246290528, -11.528040246290528, -11.528040246290525, -11.528040246290521, -11.528040246290518, -11.483400053644058, -11.483400053644056, -11.302388549200687, -11.302388549200682, -11.302388549200682, -11.30238854920068, -11.30238854920068, -11.302388549200678, -11.302388549200678, -11.302388549200675, -11.276070971645698, -11.276070971645698, -